## Long Short Term Memory (LSTM) Model

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler

In [3]:
print("Step 1: Creating Temperature Dataset") 
data = {
"Max Temp (°C)": [38, 38, 38, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40],
"Min Temp (°C)": [22, 22, 23, 23, 24, 23, 24, 24, 23, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24]
}
df = pd.DataFrame(data)
print(df.head())


Step 1: Creating Temperature Dataset
   Max Temp (°C)  Min Temp (°C)
0             38             22
1             38             22
2             38             23
3             39             23
4             39             24


In [4]:
print("Step 2: Normalizing Data")
scaler = MinMaxScaler()
temp_scaled = scaler.fit_transform(df)
print(temp_scaled[:5])

Step 2: Normalizing Data
[[0.  0. ]
 [0.  0. ]
 [0.  0.5]
 [0.5 0.5]
 [0.5 1. ]]


In [5]:
print("Step 3: Preparing Sequences")
seq_length = 3
X_seq = [temp_scaled[i:i+seq_length] for i in range(len(temp_scaled) - seq_length)]
y_seq = [temp_scaled[i+seq_length] for i in range(len(temp_scaled) - seq_length)] 
X_seq, y_seq = np.array(X_seq), np.array(y_seq)
print("Sample X_seq:", X_seq[:2])
print("Sample y_seq:", y_seq[:2])

Step 3: Preparing Sequences
Sample X_seq: [[[0.  0. ]
  [0.  0. ]
  [0.  0.5]]

 [[0.  0. ]
  [0.  0.5]
  [0.5 0.5]]]
Sample y_seq: [[0.5 0.5]
 [0.5 1. ]]


In [6]:
print("Step 4: Building and Training LSTM Model") 
model = tf.keras.Sequential([
tf.keras.layers.LSTM(50, input_shape=(seq_length, 2)),
tf.keras.layers.Dense(2)
])
model.compile(optimizer='adam', loss='mse') 
model.fit(X_seq, y_seq, epochs=20, batch_size=8, verbose=0) 
print("Model training complete.")

Step 4: Building and Training LSTM Model


/Users/poojapatchigulla/anaconda3/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model training complete.


In [7]:
print("Step 5: Predicting Last Day Temperatures")
predicted_temp = scaler.inverse_transform(model.predict(X_seq[-1].reshape(1, seq_length, 2)))
actual_temp = scaler.inverse_transform(y_seq[-1].reshape(1, -1)) 
error = np.abs(predicted_temp - actual_temp)
print("Predicted (Max, Min):", predicted_temp[0]) 
print("Actual (Max, Min):", actual_temp[0])
print("Error (Max, Min):", error[0])

Step 5: Predicting Last Day Temperatures
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step
Predicted (Max, Min): [39.812405 24.063187]
Actual (Max, Min): [40. 24.]
Error (Max, Min): [0.18759537 0.06318665]
